### TODO: add to the requirements.txt

!pip install tweepy==3.10.0 

!pip install requests 

!pip install twython==3.9.1

### TODO: include this functionality in an independent module

In [173]:
import json
from twython import Twython, TwythonError, TwythonRateLimitError
import os
import pandas as pd

In [94]:
with open("bearer_token.txt") as tokens:
    tokens_json = json.loads(tokens.read())
    #os.environ['BEARER_TOKEN'] = token.read()[:-1]
    os.environ['CONSUMER_KEY'] = tokens_json['consumer_key']
    os.environ['CONSUMER_SECRET'] = tokens_json['consumer_secret']
    os.environ['ACESS_TOKEN'] = tokens_json['access_token']
    os.environ['ACESS_TOKEN_SECRET'] = tokens_json['access_token_secret']

#### https://github.com/twitterdev/Twitter-API-v2-sample-code (no funciona)

In [96]:
consumer_key = os.environ.get("CONSUMER_KEY")
consumer_secret = os.environ.get("CONSUMER_SECRET")
access_token = os.environ.get("ACCESS_TOKEN")
access_token_secret = os.environ.get("ACCESS_TOKEN_SECRET")

In [102]:
twitter = Twython(consumer_key, consumer_secret, access_token, access_token_secret) 

In [198]:
def timeline(user_id, max_id=None):
#       """ 
#   Loop over a user's timeline, starting at max_id. 
#   Generator.
#  
#   We can get up to 15 pages. 
#   This function loops up to 16 times to make the base case 
#   `len(tweets) == 0` trigger.
#   """
    for i in range(16):
        tweets = twitter.get_user_timeline(screen_name=user_id,
                                      max_id=max_id,
                                      count=200,
                                      trim_user=True,
                                      exclude_replies=True,
                                      include_rts=False)
        if len(tweets) > 0:  # last page should have zero results
            for tweet in tweets:
                max_id = tweet['id'] - 1
                yield tweet
        else:
            break

In [199]:
from datetime import datetime
import time

def sleep_until(ts):
 # """ Sleep until the given UTC UNIX TIMESTAMP. """
    next_time = datetime.utcfromtimestamp(int(ts))
    now = datetime.utcnow()
    offset = (next_time - now).seconds
    print("Enhancing calm. Next try: {} (Currently {})".format(next_time, now))
    print("Sleeping for {}...".format(offset))
    time.sleep(offset)
    print("Continuing...")

In [202]:
fields = ['id', 'user', 'created_at', 'text', 'lang']
df_test = pd.DataFrame()
max_id = None
while True:
    try:
        for tweet in timeline('RafaelNadal', max_id):
            sub = {k: v for k, v in tweet.items() if k in fields}
            sub['user'] = sub['user']['id']
            df_test = df_test.append(sub, ignore_index=True)
            max_id = sub['id']
        max_id = None
        break
    except TwythonRateLimitError as e:
        sleep_until(e.retry_after) # sleep until the given date
    except TwythonError as e:
        # I *think* this is caused by protected profiles. 
        # I can read some user profile info, but not the timeline
        print(e)
        print("Skipping...")
        break

In [207]:
df_test

,created_at,id,text,user,lang
0,Tue Feb 08 15:06:11 +0000 2022,1.491066e+18,Hey all @FanJoltLive is launching today w/give...,344634424.0,en
1,Thu Feb 03 16:16:15 +0000 2022,1.489272e+18,Roger. You book the practice court? Or is it m...,344634424.0,en
2,Wed Feb 02 18:46:22 +0000 2022,1.488947e+18,Thanks to the fans and to those that have supp...,344634424.0,en
3,Tue Jan 18 04:44:59 +0000 2022,1.483299e+18,"Feliz de volver a Acapulco, México 🇲🇽😉🙏 https:...",344634424.0,es
4,Sun Jan 16 04:34:24 +0000 2022,1.482572e+18,"Chris, all the best to you and get well! And I...",344634424.0,en
...,...,...,...,...,...
2576,Wed Aug 03 09:50:21 +0000 2011,9.869217e+16,"Entrenando duro para la gira americana, antes ...",344634424.0,es
2577,Tue Aug 02 21:26:37 +0000 2011,9.850500e+16,No me esperaba un primer día así!! Gracias a t...,344634424.0,es
2578,Tue Aug 02 18:30:03 +0000 2011,9.846057e+16,3 de mis primos pequeños en el entrenamiento! ...,344634424.0,es
2579,Tue Aug 02 12:52:49 +0000 2011,9.837570e+16,Hi all! I have Twitter profile! I am thrilled ...,344634424.0,en


In [206]:
len(df_test[df_test['lang']=='en'])

930